# In this notebook we would train the GRU model

In [ ]:
keep 
atr_14  , OBV

drop

'ema_cross_up', 'macd_cross_up', 'oversold_reversal', 'overbought_reversal', 'trending_market'

In [ ]:
drop_gru = [
    'open', 'high', 'low', 'typical_price', 'EMA_7', 'EMA_21', 'SMA_20', 'SMA_50',
    'vwap_24h', 'close_4h', 'bollinger_upper', 'bollinger_lower',
    'resistance_level', 'support_level', 'high_low', 'high_close', 'low_close',
    'true_range', 'volume_mean_20', 'MACD_line', 'MACD_signal',
    'bollinger_width', 'volatility_regime', 'CCI', 'stoch_%D',
    'parkinson_vol', 'ema_cross_down', 'macd_cross_down',
    'vol_spike_1_5x', 'near_upper_band', 'near_lower_band',
    'break_upper_band', 'break_lower_band', 'rsi_oversold', 'rsi_overbought',
    'above_sma20', 'above_sma50', 'ema7_above_ema21', 'macd_positive',
    'volume_breakout', 'volume_breakdown', 'stoch_overbought', 'stoch_oversold',
    'cci_overbought', 'cci_oversold', 'trending_market',
    'bullish_scenario_1', 'bullish_scenario_2', 'bullish_scenario_3',
    'bullish_scenario_4', 'bullish_scenario_5', 'bullish_scenario_6',
    'bearish_scenario_1', 'bearish_scenario_2', 'bearish_scenario_3',
    'bearish_scenario_4', 'bearish_scenario_6''ema_cross_up', 'macd_cross_up', 'oversold_reversal', 'overbought_reversal'
]

In [10]:
"""
Precision-Optimized GRU for Bitcoin Direction Prediction
=======================================================
Modified to maximize PRECISION with 2x weight to precision over recall.
Using F0.5 score (beta=0.5) which emphasizes precision.

F-beta formula:
- F2 (β=2): Emphasizes RECALL (2x weight to recall)
- F1 (β=1): Equal weight to precision and recall  
- F0.5 (β=0.5): Emphasizes PRECISION (2x weight to precision)
"""

import os
import json
import optuna
import warnings
import joblib
import time
from pathlib import Path
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score, precision_score, recall_score, fbeta_score, accuracy_score
import tensorflow as tf
from tensorflow import keras

warnings.filterwarnings("ignore")
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
tf.get_logger().setLevel('ERROR')

# Set seeds for reproducibility
SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)

# ═══════════════════════════════════════════════════════════════════════
# Configuration - PRECISION FOCUSED
# ═══════════════════════════════════════════════════════════════════════

CSV_PATH = Path(r"C:\Users\ADMIN\Desktop\Coding_projects\stock_market_prediction\Stock-Market-Prediction\data\processed\gemini_btc_with_features_4h.csv")
VAL_FRAC = 0.2

# PRECISION OPTIMIZATION: Use F0.5 score (emphasizes precision)
BETA = 0.5  # ← CHANGED: β=0.5 gives 2x weight to PRECISION
PRECISION_WEIGHT = 2.0  # Additional precision weighting

MODEL_NAME = "gru_precision_optimized.h5"
N_TRIALS = 30
TIMEOUT = 100 * 60

# Same comprehensive drop list
DROP_COLS = [
    'open', 'high', 'low', 'close',
    'typical_price', 'vwap_24h', 'close_4h',
    'EMA_7', 'EMA_21', 'SMA_20', 'SMA_50',
    'bollinger_upper', 'bollinger_lower', 'bollinger_width',
    'resistance_level', 'support_level',
    'high_low', 'high_close', 'low_close', 'true_range',
    'volume_mean_20', 'MACD_line', 'MACD_signal',
    'volatility_regime', 'CCI', 'stoch_%D', 'parkinson_vol',
    'ema_cross_down', 'macd_cross_down', 'ema_cross_up', 'macd_cross_up',
    'vol_spike_1_5x', 'near_upper_band', 'near_lower_band',
    'break_upper_band', 'break_lower_band', 'rsi_oversold', 'rsi_overbought',
    'above_sma20', 'above_sma50', 'ema7_above_ema21', 'macd_positive',
    'volume_breakout', 'volume_breakdown', 'stoch_overbought', 'stoch_oversold',
    'cci_overbought', 'cci_oversold', 'trending_market',
    'oversold_reversal', 'overbought_reversal',
    'bullish_scenario_1', 'bullish_scenario_2', 'bullish_scenario_3',
    'bullish_scenario_4', 'bullish_scenario_5', 'bullish_scenario_6',
    'bearish_scenario_1', 'bearish_scenario_2', 'bearish_scenario_3',
    'bearish_scenario_4', 'bearish_scenario_6',
    'timestamp', 'date', 'Unnamed: 0'
]

# ═══════════════════════════════════════════════════════════════════════
# GPU Configuration
# ═══════════════════════════════════════════════════════════════════════

def configure_gpu():
    """Configure GPU with proper error handling."""
    gpus = tf.config.experimental.list_physical_devices('GPU')
    if gpus:
        try:
            for gpu in gpus:
                tf.config.experimental.set_memory_growth(gpu, True)
            print(f"✅ GPU configured: {len(gpus)} device(s)")
            return True
        except RuntimeError as e:
            print(f"⚠️  GPU configuration failed: {e}")
            return False
    else:
        print("⚠️  No GPU detected, using CPU")
        return False

# ═══════════════════════════════════════════════════════════════════════
# Custom Precision-Focused Metrics
# ═══════════════════════════════════════════════════════════════════════

def precision_weighted_score(y_true, y_pred, precision_weight=2.0):
    """
    Custom metric that emphasizes precision over recall.
    
    Formula: (1 + precision_weight²) * precision * recall / (precision_weight² * precision + recall)
    This is similar to F-beta but with explicit precision weighting.
    """
    precision = precision_score(y_true, y_pred, zero_division=0)
    recall = recall_score(y_true, y_pred, zero_division=0)
    
    if precision + recall == 0:
        return 0.0
    
    # Custom precision-weighted F-score
    score = (1 + precision_weight**2) * precision * recall / (precision_weight**2 * precision + recall)
    return score

def conservative_precision_score(y_true, y_pred):
    """
    Conservative precision metric that penalizes low precision heavily.
    Combines precision with penalty for low confidence predictions.
    """
    precision = precision_score(y_true, y_pred, zero_division=0)
    recall = recall_score(y_true, y_pred, zero_division=0)
    
    # Penalty for low precision (precision < 0.6 gets heavily penalized)
    precision_penalty = max(0, (precision - 0.6) / 0.4) if precision > 0.6 else 0
    
    # Bonus for high precision (precision > 0.7 gets bonus)
    precision_bonus = max(0, (precision - 0.7) / 0.3) if precision > 0.7 else 0
    
    # Base score with precision emphasis
    base_score = precision * 0.7 + recall * 0.3  # 70% precision, 30% recall
    
    # Apply bonuses and penalties
    final_score = base_score + precision_bonus * 0.2 - (1 - precision_penalty) * 0.3
    
    return max(0, min(1, final_score))

# ═══════════════════════════════════════════════════════════════════════
# Data Loading and Preprocessing
# ═══════════════════════════════════════════════════════════════════════

def load_and_preprocess_data():
    """Load and preprocess data with comprehensive validation."""
    
    print("🚀 Loading Bitcoin data for PRECISION-optimized GRU...")
    print("=" * 60)
    
    if not CSV_PATH.exists():
        raise FileNotFoundError(f"❌ Data file not found: {CSV_PATH}")
    
    df = pd.read_csv(CSV_PATH, index_col=0, parse_dates=True)
    print(f"✅ Loaded data: {df.shape}")
    
    # Filter to stable period
    df = df[df.index >= "2020-01-01"]
    print(f"📅 Using data from 2020+: {len(df)} rows")
    
    if 'target' not in df.columns:
        raise ValueError("❌ Target column not found!")
    
    target_dist = df['target'].value_counts().to_dict()
    print(f"🎯 Target distribution: {target_dist}")
    
    # Drop columns
    cols_to_drop = [c for c in DROP_COLS if c in df.columns]
    df = df.drop(columns=cols_to_drop)
    df = df[df["target"].notna()].dropna()
    print(f"🧹 After cleaning: {df.shape}")
    
    # Validate no data leakage
    features = df.drop(columns="target")
    forbidden_cols = ['open', 'high', 'low', 'close']
    found_forbidden = [col for col in forbidden_cols if col in features.columns]
    
    if found_forbidden:
        raise ValueError(f"❌ DATA LEAKAGE: {found_forbidden} in features!")
    
    print(f"✅ Features ({len(features.columns)}): {list(features.columns)}")
    print(f"✅ PRECISION OPTIMIZATION: No price data leakage")
    
    target = df["target"].astype(int).values
    
    if len(df) < 1000:
        raise ValueError(f"❌ Insufficient data: {len(df)} rows")
    
    return features, target, df.index

# ═══════════════════════════════════════════════════════════════════════
# Sequence Creation
# ═══════════════════════════════════════════════════════════════════════

def create_sequences(data, labels, window_size):
    """Create sequences with validation."""
    if window_size >= len(data):
        raise ValueError(f"Window {window_size} >= data length {len(data)}")
    
    X, y = [], []
    for i in range(window_size, len(data)):
        X.append(data[i-window_size:i])
        y.append(labels[i])
    
    return np.array(X, dtype=np.float32), np.array(y, dtype=np.int8)

# ═══════════════════════════════════════════════════════════════════════
# Precision-Optimized Objective Function
# ═══════════════════════════════════════════════════════════════════════

def precision_objective(trial, data_info):
    """
    Objective function optimized for PRECISION.
    Uses multiple precision-focused metrics.
    """
    
    X_train_scaled, X_val_scaled, y_train, y_val = data_info
    
    try:
        # Sample hyperparameters (slightly more conservative for precision)
        window_size = trial.suggest_int("window", 12, 48, step=6)
        n_units = trial.suggest_int("units", 32, 128, step=32)
        n_layers = trial.suggest_int("layers", 1, 2)
        dropout_rate = trial.suggest_float("dropout", 0.2, 0.5)  # Higher dropout for precision
        learning_rate = trial.suggest_float("lr", 1e-4, 5e-3, log=True)
        l2_reg = trial.suggest_float("l2", 1e-5, 1e-2, log=True)  # More regularization
        batch_size = trial.suggest_categorical("batch", [32, 64])
        
        # PRECISION OPTIMIZATION: Threshold tuning
        decision_threshold = trial.suggest_float("threshold", 0.4, 0.7)  # Conservative thresholds
        
        # Create sequences
        X_train_seq, y_train_seq = create_sequences(X_train_scaled, y_train, window_size)
        X_val_seq, y_val_seq = create_sequences(X_val_scaled, y_val, window_size)
        
        if len(X_train_seq) < 100 or len(X_val_seq) < 50:
            return float('inf')
        
        if len(np.unique(y_train_seq)) < 2 or len(np.unique(y_val_seq)) < 2:
            return float('inf')
        
        # Build model (more conservative architecture for precision)
        tf.keras.backend.clear_session()
        model = keras.Sequential()
        
        for i in range(n_layers):
            return_sequences = (i < n_layers - 1)
            model.add(keras.layers.GRU(
                n_units,
                return_sequences=return_sequences,
                dropout=dropout_rate,
                recurrent_dropout=dropout_rate * 0.6,  # Higher recurrent dropout
                kernel_regularizer=keras.regularizers.l2(l2_reg),
                recurrent_regularizer=keras.regularizers.l2(l2_reg)
            ))
        
        # Add extra regularization for precision
        model.add(keras.layers.Dropout(dropout_rate))
        model.add(keras.layers.Dense(1, activation="sigmoid"))
        
        # Compile model
        model.compile(
            optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
            loss="binary_crossentropy",
            metrics=['accuracy']
        )
        
        # Train model with patience for precision
        history = model.fit(
            X_train_seq, y_train_seq,
            epochs=80,
            batch_size=batch_size,
            validation_data=(X_val_seq, y_val_seq),
            callbacks=[
                keras.callbacks.EarlyStopping(
                    monitor='val_loss',
                    patience=15,  # More patience for precision optimization
                    restore_best_weights=True,
                    verbose=0
                )
            ],
            verbose=0
        )
        
        # Evaluate with custom threshold
        y_pred_prob = model.predict(X_val_seq, verbose=0).ravel()
        y_pred = (y_pred_prob >= decision_threshold).astype(int)
        
        # Calculate precision-focused metrics
        precision = precision_score(y_val_seq, y_pred, zero_division=0)
        recall = recall_score(y_val_seq, y_pred, zero_division=0)
        f1 = f1_score(y_val_seq, y_pred, zero_division=0)
        f_half = fbeta_score(y_val_seq, y_pred, beta=BETA, zero_division=0)  # F0.5 (precision-focused)
        accuracy = accuracy_score(y_val_seq, y_pred)
        
        # Custom precision metrics
        precision_weighted = precision_weighted_score(y_val_seq, y_pred, PRECISION_WEIGHT)
        conservative_precision = conservative_precision_score(y_val_seq, y_pred)
        
        # Store all metrics
        trial.set_user_attr("precision", precision)
        trial.set_user_attr("recall", recall)
        trial.set_user_attr("f1_score", f1)
        trial.set_user_attr("f_half_score", f_half)  # F0.5 score
        trial.set_user_attr("accuracy", accuracy)
        trial.set_user_attr("precision_weighted", precision_weighted)
        trial.set_user_attr("conservative_precision", conservative_precision)
        trial.set_user_attr("decision_threshold", decision_threshold)
        trial.set_user_attr("n_positive_predictions", int(np.sum(y_pred)))
        
        # PRECISION OPTIMIZATION: Multiple scoring strategies
        
        # Strategy 1: Pure F0.5 (emphasizes precision)
        score_f_half = f_half
        
        # Strategy 2: Precision-weighted custom score
        score_custom = precision_weighted
        
        # Strategy 3: Conservative precision score
        score_conservative = conservative_precision
        
        # Strategy 4: Precision with minimum recall constraint
        if recall >= 0.2:  # Minimum 20% recall required
            score_constrained = precision
        else:
            score_constrained = precision * (recall / 0.2)  # Penalty for low recall
        
        # Combine strategies (weighted average)
        final_score = (
            0.4 * score_f_half +           # 40% F0.5 score
            0.3 * score_custom +           # 30% custom precision score  
            0.2 * score_conservative +     # 20% conservative score
            0.1 * score_constrained        # 10% constrained precision
        )
        
        trial.set_user_attr("final_combined_score", final_score)
        
        # Return negative score for minimization
        return 1.0 - final_score
        
    except Exception as e:
        print(f"❌ Trial {trial.number} failed: {e}")
        return float('inf')

# ═══════════════════════════════════════════════════════════════════════
# Enhanced Progress Callback
# ═══════════════════════════════════════════════════════════════════════

def precision_progress_callback(study, trial):
    """Progress callback focused on precision metrics."""
    
    if trial.state == optuna.trial.TrialState.COMPLETE:
        attrs = trial.user_attrs
        precision = attrs.get('precision', 0)
        recall = attrs.get('recall', 0)
        f_half = attrs.get('f_half_score', 0)
        threshold = attrs.get('decision_threshold', 0.5)
        n_pos_pred = attrs.get('n_positive_predictions', 0)
        
        best_score = 1 - study.best_value if study.best_value != float('inf') else 0
        
        print(f"Trial {trial.number:2d}: P={precision:.3f} R={recall:.3f} F0.5={f_half:.3f} "
              f"Thr={threshold:.2f} Pos={n_pos_pred} | Best={best_score:.3f}")
    
    elif trial.state == optuna.trial.TrialState.PRUNED:
        print(f"Trial {trial.number:2d}: PRUNED")

# ═══════════════════════════════════════════════════════════════════════
# Main Precision Optimization Pipeline
# ═══════════════════════════════════════════════════════════════════════

if __name__ == "__main__":
    print("🚀 PRECISION-OPTIMIZED GRU HYPERPARAMETER SEARCH")
    print("🎯 Maximizing PRECISION for Bitcoin Direction Prediction")
    print("=" * 70)
    print(f"📊 Using F{BETA} score (β={BETA}) - Emphasizes PRECISION")
    print(f"🎯 Precision gets {PRECISION_WEIGHT}x weight in scoring")
    
    try:
        # Configure GPU
        gpu_available = configure_gpu()
        
        # Load and preprocess data
        features, target, dates = load_and_preprocess_data()
        
        # Proper scaling
        split_idx = int(len(features) * (1 - VAL_FRAC))
        
        scaler = StandardScaler()
        scaler.fit(features.iloc[:split_idx])
        
        X_train_scaled = scaler.transform(features.iloc[:split_idx])
        X_val_scaled = scaler.transform(features.iloc[split_idx:])
        y_train = target[:split_idx]
        y_val = target[split_idx:]
        
        print(f"📊 Train: {len(X_train_scaled)} samples, Val: {len(X_val_scaled)} samples")
        
        # Save scaler
        joblib.dump(scaler, "gru_precision_scaler.pkl")
        
        # Prepare data
        data_info = (X_train_scaled, X_val_scaled, y_train, y_val)
        
        # Create study
        study = optuna.create_study(
            direction="minimize",
            sampler=optuna.samplers.TPESampler(seed=SEED),
            pruner=optuna.pruners.MedianPruner(
                n_startup_trials=5,
                n_warmup_steps=3
            ),
            study_name="gru_precision_optimized"
        )
        
        # Run optimization
        print(f"\n🔍 Starting PRECISION optimization...")
        print(f"   Focus: Maximize precision with 2x weight")
        print(f"   Trials: {N_TRIALS}, Timeout: {TIMEOUT//60}min")
        
        start_time = time.time()
        
        study.optimize(
            lambda trial: precision_objective(trial, data_info),
            n_trials=N_TRIALS,
            timeout=TIMEOUT,
            show_progress_bar=True,
            callbacks=[precision_progress_callback]
        )
        
        total_time = time.time() - start_time
        
        # Results analysis
        print(f"\n🏆 PRECISION OPTIMIZATION COMPLETED")
        print("=" * 50)
        
        completed_trials = [t for t in study.trials if t.state == optuna.trial.TrialState.COMPLETE]
        
        if completed_trials:
            best_combined_score = 1 - study.best_value
            best_trial = study.best_trial
            
            print(f"⭐ Best combined score: {best_combined_score:.4f}")
            print(f"📊 Completed trials: {len(completed_trials)}")
            print(f"⏱️  Total time: {total_time/60:.1f} minutes")
            
            # Best trial precision metrics
            if hasattr(best_trial, 'user_attrs'):
                attrs = best_trial.user_attrs
                print(f"\n🎯 BEST TRIAL PRECISION METRICS:")
                print(f"   Precision: {attrs.get('precision', 'N/A'):.4f} ⭐")
                print(f"   Recall: {attrs.get('recall', 'N/A'):.4f}")
                print(f"   F0.5-score: {attrs.get('f_half_score', 'N/A'):.4f} (precision-focused)")
                print(f"   F1-score: {attrs.get('f1_score', 'N/A'):.4f}")
                print(f"   Decision threshold: {attrs.get('decision_threshold', 'N/A'):.3f}")
                print(f"   Positive predictions: {attrs.get('n_positive_predictions', 'N/A')}")
            
            print(f"\n🎯 BEST PARAMETERS:")
            for param, value in study.best_params.items():
                print(f"   {param:15s}: {value}")
            
            # Save results
            enhanced_params = {
                **study.best_params,
                "optimization_focus": "precision",
                "beta_score": BETA,
                "precision_weight": PRECISION_WEIGHT,
                "best_metrics": {
                    k: float(v) if isinstance(v, (int, float, np.number)) else v
                    for k, v in best_trial.user_attrs.items()
                    if k in ['precision', 'recall', 'f_half_score', 'f1_score', 'decision_threshold']
                }
            }
            
            with open("gru_precision_optimized_params.json", "w") as f:
                json.dump(enhanced_params, f, indent=2)
            
            print(f"\n✅ Precision-optimized parameters saved!")
            print(f"🎯 Model optimized for HIGH PRECISION trading signals")
            
        else:
            print("❌ No trials completed successfully!")
    
    except Exception as e:
        print(f"❌ Optimization failed: {e}")
        import traceback
        traceback.print_exc()

[I 2025-06-07 12:55:45,523] A new study created in memory with name: gru_precision_optimized


🚀 PRECISION-OPTIMIZED GRU HYPERPARAMETER SEARCH
🎯 Maximizing PRECISION for Bitcoin Direction Prediction
📊 Using F0.5 score (β=0.5) - Emphasizes PRECISION
🎯 Precision gets 2.0x weight in scoring
⚠️  No GPU detected, using CPU
🚀 Loading Bitcoin data for PRECISION-optimized GRU...
✅ Loaded data: (20718, 66)
📅 Using data from 2020+: 11476 rows
🎯 Target distribution: {1: 5858, 0: 5618}
🧹 After cleaning: (11476, 20)
✅ Features (19): ['volume', 'RSI', 'MACD_histogram', 'OBV', 'stoch_%K', 'atr_14', 'atr_ratio', 'price_vs_vwap', 'volume_ratio', 'buying_pressure', 'adx', 'fear_greed_score', 'roc_4h', 'roc_24h', 'bb_position', 'macd_rising', 'obv_rising_24h', 'momentum_alignment', 'trend_alignment']
✅ PRECISION OPTIMIZATION: No price data leakage
📊 Train: 9180 samples, Val: 2296 samples

🔍 Starting PRECISION optimization...
   Focus: Maximize precision with 2x weight
   Trials: 30, Timeout: 100min


Best trial: 0. Best value: 0.981382:   3%|▎         | 1/30 [06:53<3:19:57, 413.71s/it, 413.71/6000 seconds]

[I 2025-06-07 13:02:39,228] Trial 0 finished with value: 0.9813824931254083 and parameters: {'window': 24, 'units': 128, 'layers': 2, 'dropout': 0.379597545259111, 'lr': 0.00018410729205738696, 'l2': 2.9375384576328295e-05, 'batch': 64, 'threshold': 0.5803345035229626}. Best is trial 0 with value: 0.9813824931254083.
Trial  0: P=0.409 R=0.008 F0.5=0.036 Thr=0.58 Pos=22 | Best=0.019


Best trial: 0. Best value: 0.981382:   7%|▋         | 2/30 [10:45<2:23:00, 306.45s/it, 645.08/6000 seconds]

...

[I 2025-06-07 14:23:39,902] Trial 27 finished with value: 0.3919346379287192 and parameters: {'window': 36, 'units': 64, 'layers': 1, 'dropout': 0.34882690126287674, 'lr': 0.0007750288765009825, 'l2': 0.0007786020101904646, 'batch': 32, 'threshold': 0.42606199161643293}. Best is trial 22 with value: 0.3918097792402703.
Trial 27: P=0.520 R=1.000 F0.5=0.575 Thr=0.43 Pos=2260 | Best=0.608


Best trial: 22. Best value: 0.39181:  97%|█████████▋| 29/30 [1:28:31<01:23, 83.40s/it, 5311.20/6000 seconds] 

[I 2025-06-07 14:24:16,725] Trial 28 finished with value: 0.39240567982827246 and parameters: {'window': 18, 'units': 32, 'layers': 1, 'dropout': 0.3454456702065626, 'lr': 0.003010191223030362, 'l2': 0.0002696124550195139, 'batch': 32, 'threshold': 0.4640763508312489}. Best is trial 22 with value: 0.3918097792402703.
Trial 28: P=0.519 R=1.000 F0.5=0.575 Thr=0.46 Pos=2278 | Best=0.608


Best trial: 22. Best value: 0.39181: 100%|██████████| 30/30 [1:29:41<00:00, 179.40s/it, 5381.86/6000 seconds]

[I 2025-06-07 14:25:27,379] Trial 29 finished with value: 1.0 and parameters: {'window': 30, 'units': 64, 'layers': 1, 'dropout': 0.37182564519122707, 'lr': 0.001233010707619071, 'l2': 5.347995878462169e-05, 'batch': 32, 'threshold': 0.5989922993489152}. Best is trial 22 with value: 0.3918097792402703.
Trial 29: P=0.000 R=0.000 F0.5=0.000 Thr=0.60 Pos=0 | Best=0.608

🏆 PRECISION OPTIMIZATION COMPLETED
⭐ Best combined score: 0.6082
📊 Completed trials: 30
⏱️  Total time: 89.7 minutes

🎯 BEST TRIAL PRECISION METRICS:
   Precision: 0.5215 ⭐
   Recall: 0.9957
   F0.5-score: 0.5764 (precision-focused)
   F1-score: 0.6845
   Decision threshold: 0.445
   Positive predictions: 2236

🎯 BEST PARAMETERS:
   window         : 42
   units          : 96
   layers         : 1
   dropout        : 0.29304784026893504
   lr             : 0.0009539147035893642
   l2             : 0.0006182071520515096
   batch          : 32
   threshold      : 0.4449592913770786

✅ Precision-optimized parameters saved!
🎯 M

In [12]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from pathlib import Path
from sklearn.preprocessing import StandardScaler

# ─── Config ───────────────────────────────────────────────
CSV_PATH = Path(r"C:\Users\ADMIN\Desktop\Coding_projects\stock_market_prediction"
                r"\Stock-Market-Prediction\data\processed\gemini_btc_with_features_4h.csv")
MODEL_NAME = "gru_best_model_val20.h5"
VAL_FRAC = 0.20
SEED = 42

# ─── Best Parameters from Optuna ──────────────────────────
BEST_PARAMS = {
    'window': 48,
    'units': 96,
    'layers': 1,
    'dropout': 0.2970094826765549,
    'lr': 0.001278412165651303,
    'l2': 0.007238819361166793,
    'batch': 32
}

np.random.seed(SEED)
tf.random.set_seed(SEED)

# ─── Load & Preprocess Data ───────────────────────────────
df = pd.read_csv(CSV_PATH, index_col=0, parse_dates=True)
df = df[df.index >= "2018-01-01"]
drop_cols = [  # must match your Optuna script
    'open', 'high', 'low', 'close', 'typical_price', 'EMA_7', 'EMA_21', 'SMA_20', 'SMA_50',
    'vwap_24h', 'close_4h', 'bollinger_upper', 'bollinger_lower', 'resistance_level',
    'support_level', 'high_low', 'high_close', 'low_close', 'true_range', 'volume_mean_20',
    'MACD_line', 'MACD_signal', 'bollinger_width', 'volatility_regime', 'CCI', 'stoch_%D',
    'parkinson_vol', 'ema_cross_down', 'macd_cross_down', 'vol_spike_1_5x', 'near_upper_band',
    'near_lower_band', 'break_upper_band', 'break_lower_band', 'rsi_oversold', 'rsi_overbought',
    'above_sma20', 'above_sma50', 'ema7_above_ema21', 'macd_positive', 'volume_breakout',
    'volume_breakdown', 'stoch_overbought', 'stoch_oversold', 'cci_overbought', 'cci_oversold',
    'trending_market', 'bullish_scenario_1', 'bullish_scenario_2', 'bullish_scenario_3',
    'bullish_scenario_4', 'bullish_scenario_5', 'bullish_scenario_6', 'bearish_scenario_1',
    'bearish_scenario_2', 'bearish_scenario_3', 'bearish_scenario_4', 'bearish_scenario_6',
    'ema_cross_up', 'macd_cross_up', 'oversold_reversal', 'overbought_reversal'
]
df.drop(columns=[c for c in drop_cols if c in df.columns], inplace=True)
df = df[df["target"].notna()].dropna()

features = df.drop(columns="target")
target = df["target"].astype(int).values

scaler = StandardScaler()
features_scaled = scaler.fit_transform(features)

split_idx = int(len(df) * (1 - VAL_FRAC))
X_train_raw, X_val_raw = features_scaled[:split_idx], features_scaled[split_idx:]
y_train, y_val = target[:split_idx], target[split_idx:]

# ─── Sequence Windows ─────────────────────────────────────
def make_windows(data, labels, win):
    X, y = [], []
    for i in range(win, len(data)):
        X.append(data[i-win:i])
        y.append(labels[i])
    return np.array(X, dtype=np.float32), np.array(y, dtype=np.int8)

X_tr, y_tr = make_windows(X_train_raw, y_train, BEST_PARAMS["window"])
X_va, y_va = make_windows(X_val_raw, y_val, BEST_PARAMS["window"])

# ─── Build and Train GRU Model ────────────────────────────
tf.keras.backend.clear_session()
model = tf.keras.Sequential()
for i in range(BEST_PARAMS["layers"]):
    model.add(tf.keras.layers.GRU(
        BEST_PARAMS["units"],
        return_sequences=(i < BEST_PARAMS["layers"] - 1),
        dropout=BEST_PARAMS["dropout"],
        kernel_regularizer=tf.keras.regularizers.l2(BEST_PARAMS["l2"])
    ))
model.add(tf.keras.layers.Dense(1, activation="sigmoid"))

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=BEST_PARAMS["lr"]),
    loss="binary_crossentropy"
)

model.fit(
    X_tr, y_tr,
    epochs=50,
    batch_size=BEST_PARAMS["batch"],
    validation_data=(X_va, y_va),
    verbose=0
)

model.save(MODEL_NAME)
print(f"\n✔ GRU model saved → {MODEL_NAME}")



✔ GRU model saved → gru_best_model_val20.h5


In [ ]:
🧪 Training GRU #1 with params: {'window': 42, 'units': 96, 'layers': 1, 'dropout': 0.39398, 'lr': 0.001585, 'l2': 0.000266, 'batch': 64}
[1m98/98[0m [32m━━━━━━━━━━━━━━━━━━━━[0m[37m[0m [1m1s[0m 7ms/step
✅ GRU #1 Results:
   Precision: 0.522
   Recall   : 0.425
   F1-score : 0.469
   F2-score : 0.441
   Accuracy : 0.497

🧪 Training GRU #2 with params: {'window': 36, 'units': 64, 'layers': 1, 'dropout': 0.26507, 'lr': 0.000857, 'l2': 0.000349, 'batch': 32}
[1m98/98[0m [32m━━━━━━━━━━━━━━━━━━━━[0m[37m[0m [1m1s[0m 5ms/step
✅ GRU #2 Results:
   Precision: 0.517
   Recall   : 0.477
   F1-score : 0.496
   F2-score : 0.485
   Accuracy : 0.494

🧪 Training GRU #3 with params: {'window': 36, 'units': 96, 'layers': 1, 'dropout': 0.25825, 'lr': 0.001168, 'l2': 0.00019, 'batch': 64}
[1m98/98[0m [32m━━━━━━━━━━━━━━━━━━━━[0m[37m[0m [1m2s[0m 17ms/step
✅ GRU #3 Results:
   Precision: 0.514
   Recall   : 0.443
   F1-score : 0.476
   F2-score : 0.456
   Accuracy : 0.491

In [ ]:
🧪 Training GRU #2 with params: {'window': 36, 'units': 96, 'layers': 2, 'dropout': 0.297, 'lr': 0.001278, 'l2': 0.007239, 'batch': 32}
98/98 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step
✅ GRU #2 Results:
   Precision: 0.522
   Recall   : 1.000
   F1-score : 0.686
   F2-score : 0.845
   Accuracy : 0.522